# BiteWise

From pantry chaos to plated perfection—smarter, healthier, waste-wise cooking starts here.

### 📢 Additional Materials
📄 Project Blog Post: [BiteWise: Revolutionizing Meal Planning](https://medium.com/@srastegarnia/bitewise-revolutionizing-meal-planning-with-your-ai-kitchen-assistant-fa76724b6ce9)

🎥 Demo Video: [Watch on YouTube](https://www.youtube.com/watch?v=bUXtz69sTOs&t=7s)

# Overview
This project recommends personalized recipes based on users' available ingredients, specified calorie limits, dietary preferences, chosen cuisines, allergies, and pantry contents.

#### **Key Functionalities:**
* **Ingredient-based Recipe Suggestions:** Turn your existing pantry and fridge contents into delicious meals without additional shopping trips.
* **Calorie-Conscious Recommendations:** Automatically filter and suggest recipes that align with your specified calorie goals, making healthier eating effortless.
* **Dietary Customization and allergies:** Filters recipes according to dietary restrictions (e.g., vegan, keto, gluten-free) and specific allergies (e.g., nuts, dairy).
* **Global Cuisine Exploration:** Access recipes from diverse culinary traditions based on your preference, encouraging culinary adventure while using familiar ingredients.
* **Detailed Recipe Information:** Enables users to explore suggested recipes with full preparation instructions and nutritional breakdowns.
* **Photo and Manual Ingredient Input:** Supports image uploads of fridge/pantry contents to improve ingredient detection accuracy through image recognition. Users can also manually add ingredients for greater flexibility.


# Problem Statement

In today’s world, people struggle to make the meals they want due to limited time, leftover ingredients that lead to food waste, and personal constraints like health goals, dietary rules, allergies, and favourite cuisines that narrow their options. They need a quick way to decide what to cook without scrolling through countless online recipes.

This project addresses this gap by providing a solution that:
* Converts the user’s current pantry, either manually or through a photo, into the primary search space.
* Filters candidate dishes based on calories, allergies, and dietary rules in a single pass.
* Delivers rapid, clear and straightforward suggestions to help users cook meals, save time, and reduce food waste.



# BiteWise Users 
* **Home Cooks** – Make meals using what’s already in the kitchen.

* **Health-Conscious Eaters** – Track calories and meet fitness or wellness goals.

* **Diet-Restricted Users** – Follow specific diets (vegan, keto, gluten-free) or avoid allergens.

* **Culinary Explorers** – Try new global recipes using familiar ingredients.

* **Busy Individuals** – Need quick, simple recipes with minimal prep.

* **Sustainability-Focused Users** – Reduce food waste and cook with seasonal or local items.

# Gen AI Capabilities
BiteWise leverages cutting-edge generative AI technologies to deliver personalized recipe recommendations:
*  **JSON mode**: Implements structured JSON output formatting to generate consistently parseable recipe data containing standardized fields for names, ingredients, and precise calorie estimates.
* **Few-shot prompting**: Utilizes advanced example-based instruction techniques to guide the AI in generating contextually appropriate, calorie-conscious recipe suggestions that precisely account for selected ingredients, allergies, dietary needs, and preferred cuisines. This approach significantly improves the quality and relevance of recommendations compared to zero-shot prompting.
* **Image understanding**: Employs computer vision algorithms to analyze user-provided images of grocery items, accurately identifying available ingredients while filtering out non-food objects. This visual recognition system enables a frictionless user experience by automatically converting refrigerator/pantry photos into usable ingredient lists.


## Import packages

In [ ]:
from IPython.display import Markdown, display, Image
import typing_extensions as typing
import io
import base64
from PIL import Image as PILImage
import ipywidgets as widgets
import google.generativeai as genai
import json

## Data Schemas  
Specify the data structures for representing individual recipes and recipe collections.

In [ ]:
class Recipe(typing.TypedDict):
    """Define the recipe schemas"""
    
    name: str
    ingredients: list[str]
    estimated_calories: int



class RecipeList(typing.TypedDict): 
    """Schema for a collection of recipes."""
    recipes: list[Recipe]

## Ingredient Recognition System

The vision-based ingredient detection system:

* Processes user-uploaded refrigerator/pantry images
* Applies computer vision algorithms to identify visible food items
* Filters out non-food objects (containers, utensils) for accurate ingredient listing
* Returns a clean, structured list of available cooking ingredients


In [ ]:
def display_image(image_bytes):
    """Helper function to display uploaded images"""
    
    image = PILImage.open(io.BytesIO(image_bytes))
    return image

def identify_ingredients_from_image(client, image_bytes):
    """
    Identifies the list of ingredients in an image.
    Args:
        client: The API client used to generate content.
        image_bytes: Image data.
    """

    # Instruct the AI to list each visible raw food ingredient on its own line
    prompt = """
    Please identify all food ingredients visible in this image.
    Focus only on raw ingredients, produce, and food items.
    Ignore any containers, utensils, or non-food items.
    List each identified ingredient on a new line.
    """
    image_part = {
        "inline_data": {
            "mime_type": "image/jpeg",
            "data": base64.b64encode(image_bytes).decode()
        }
    }
    
    # Send the image and prompt to the API
    response = client.generate_content(
        contents=[{"parts": [{"text": prompt}, image_part]}]
    )
    
    ingredients_text = response.text.strip()
    ingredients_list = [ingredient.strip() for ingredient in ingredients_text.split('\n') if ingredient.strip() and not ingredient.lower().startswith("okay") and not ingredient.lower().startswith("here are")]
    
    return ingredients_list

## Recipe Recommendation Engine
This cell generates recipe suggestions by combining the user’s available ingredients and pantry with optional constraints such as calorie limits, cuisine preferences, allergies, and dietary restrictions. It constructs a few-shot prompt that includes examples of various recipe scenarios to produce relevant outputs in JSON format. The result contains recipes customized to the provided context, with each suggestion listing the name, ingredients used, and estimated calories.

In [ ]:
def suggest_recipes_from_ingredients(client, ingredients, calorie_limit, cuisines=None, allergies=None, diets=None):
    """
    Suggests recipes based on a list of ingredients, calorie limit, cuisine, allergies, and dietary preferences.
    Args:
        client: The API client used to generate content.
        ingredients: A list of available ingredients.
        calorie_limit: The maximum calorie limit for the suggested recipes (integer, e.g., 500).
        cuisines: (Optional) A list of preferred cuisines (e.g., ["Italian", "Mexican"]).
        allergies: (Optional) A list of allergens to avoid (e.g., ["nuts", "lactose"]).
        diets: (Optional) A list of dietary preferences (e.g., ["vegetarian", "keto"]).
    """

    # Convert lists to comma-separated strings 
    ingredients_str = ", ".join(ingredients)
    cuisines_str = ", ".join(cuisines) if cuisines else "any cuisine"
    allergies_str = ", ".join(allergies) if allergies else "none"
    diets_str = ", ".join(diets) if diets else "none"

    # Few-shot examples
    few_shot_prompt = f"""
    You are a Waste and Calorie Wise Food Crafter.
    Suggest recipes based on available ingredients, following these rules:
    - Only use the user's provided ingredients: {ingredients_str}.
    - Allowed extras (if not provided):
        - For **Keto**: butter, olive oil, apple cider vinegar, salt, pepper, water.
        - For **other diets**: salt, pepper, oil, vinegar, water. 
    - Prioritize low waste (use all items if possible).
    - Stay strictly at or under the calorie limit ({calorie_limit} kcal).
    - Match preferred cuisines when possible ({cuisines_str}).
    - Respect dietary preferences: {diets_str}.
    - Avoid allergens: {allergies_str}.
    - Return a JSON object with recipe names, ingredients used, and estimated calories.

    ### **Keto-Specific Rules (if selected):**  
    1. **Prioritize**: Meat/Fish/Eggs > Low-carb veggies (zucchini, spinach, broccoli).  
    2. **Macros**: High fat, moderate protein, minimal carbs (<5% of calories).  
    3. **Banned**: Grains, starchy veggies, legumes, sugar. 
    

    Example Input:
    Ingredients: rice, chicken, soy sauce
    Calorie Limit: 500 kcal
    Cuisines: Chinese
    Allergies: none
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Soy Chicken Fried Rice",
    "ingredients": ["rice", "chicken", "soy sauce"],
    "estimated_calories": 450}},
    {{"name": "Chicken Rice Stir-Fry",
    "ingredients": ["rice", "chicken", "soy sauce"],
    "estimated_calories": 480}}
    ]
    }}

    Example Input:
    Ingredients: spinach, canned tuna, lemon
    Calorie Limit: 300 kcal
    Cuisines: Italian
    Allergies: none
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Lemon Tuna Spinach Salad",
    "ingredients": ["spinach", "canned tuna", "lemon"],
    "estimated_calories": 250}},
    {{"name": "Spinach Tuna Citrus Mix",
    "ingredients": ["spinach", "canned tuna", "lemon"],
    "estimated_calories": 260}}
    ]
    }}

    Example Input:
    Ingredients: tomatoes, black beans, corn, avocado, chicken
    Calorie Limit: 400 kcal
    Cuisines: Mexican
    Allergies: Nuts, Lactose
    Dietary Preferences: Vegetarian
    Example Output:
    {{
    "recipes": [
    {{"name": "Black Bean and Corn Salsa",
    "ingredients": ["tomatoes", "black beans", "corn"],
    "estimated_calories": 300}},
    {{"name": "Avocado Tomato Salad",
    "ingredients": ["tomatoes", "avocado"],
    "estimated_calories": 350}}
    ]
    }}

    Example Input:
    Ingredients: rice, chicken, soy sauce, pasta, beans
    Calorie Limit: 600 kcal
    Cuisines: Chinese
    Allergies: Beans
    Dietary Preferences: Gluten-free
    Example Output:
    {{
    "recipes": [
    {{"name": "Gluten-Free Soy Chicken Fried Rice",
    "ingredients": ["rice", "chicken", "gluten-free soy sauce"],
    "estimated_calories": 450}},
    {{"name": "Chicken Rice Stir-Fry",
    "ingredients": ["rice", "chicken", "gluten-free soy sauce"],
    "estimated_calories": 450}}
    ]
    }}

    Example Input:
    Ingredients: sweet potatoes, kale, chickpeas, tahini, spinach, onion, garlic, ginger, mustard seeds
    Calorie Limit: 400 kcal
    Cuisines: Indian
    Allergies: Gluten
    Dietary Preferences: Vegan
    Example Output:
    {{
    "recipes": [
    {{"name": "Sweet Potato and Chickpea Curry (Gluten-Free)",
    "ingredients": ["sweet potatoes", "spinach", "chickpeas", "tahini", "onion", "garlic"],
    "estimated_calories": 390}},
    {{"name": "Spiced Chickpea and Sweet Potato Stir-Fry",
    "ingredients": ["sweet potatoes", "chickpeas", "kale", "onion", "ginger","mustard seeds"],
    "estimated_calories": 360}}
    ]
    }}

    Example Input:
    Ingredients: salmon, asparagus, quinoa, lemon, garlic
    Calorie Limit: 500 kcal
    Cuisines: American
    Allergies: Dairy
    Dietary Preferences: Pescatarian
    Example Output:
    {{
    "recipes": [
    {{"name": "Lemon Garlic Salmon with Quinoa and Asparagus",
    "ingredients": ["salmon", "asparagus", "quinoa", "lemon", "garlic"],
    "estimated_calories": 480}},
    {{"name": "Quinoa Asparagus Salad with Lemon Salmon",
    "ingredients": ["salmon", "asparagus", "quinoa", "lemon"],
    "estimated_calories": 450}}
    ]
    }}

    Example Input:
    Ingredients: rice, Canned tuna in water, light mayo, Soy sauce, Nori seaweed, Sesame seeds, Pickled ginger,Cucumber 
    Calorie Limit: 400 kcal
    Cuisines: Japanese
    Allergies: none
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Tuna Onigiri(Rice Ball)",
    "ingredients": ["rice", "Canned tuna in water", "light mayo", "Soy sauce", "Nori seaweed", "Sesame seeds", "Pickled ginger"],
    "estimated_calories": 380}},
    {{"name": "Tuna cucmber Temaki",
    "ingredients": ["rice", "Canned tuna in water", "Soy sauce", "Nori seaweed", "Cucumber", "Sesame seeds", "Pickled ginger"],
    "estimated_calories": 370}}
    ]
    }}

    Example Input:
    Ingredients: zucchini, ground turkey, tomatoes, onions
    Calorie Limit: 350 kcal
    Cuisines: Italian
    Allergies: none
    Dietary Preferences: Keto
    Example Output:
    {{
    "recipes": [
    {{"name": "Keto Turkey Zucchini Skillet",
    "ingredients": ["zucchini", "ground turkey", "tomatoes", "onions"],
    "estimated_calories": 320}},
    {{"name": "Italian Turkey Zucchini Boats",
    "ingredients": ["zucchini", "ground turkey", "tomatoes"],
    "estimated_calories": 300}}
    ]
    }}

    Example Input:
    Ingredients: chicken thighs, rice, coconut milk, curry paste, onions
    Calorie Limit: 800 kcal
    Cuisines: Indian
    Allergies: none
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Coconut Chicken Curry with Rice",
    "ingredients": ["chicken thighs", "rice", "coconut milk", "curry paste", "onions"],
      "estimated_calories": 750}},
    {{"name": "Spicy Chicken Curry Rice Bowl",
    "ingredients": ["chicken thighs", "rice", "coconut milk", "curry paste"],
    "estimated_calories": 720}}
    ]
    }}

    Example Input:
    Ingredients: ground beef, pasta, tomatoes, cheese, cream
    Calorie Limit: 900 kcal
    Cuisines: Italian
    Allergies: none
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Creamy Beef Pasta Bake",
    "ingredients": ["ground beef", "pasta", "tomatoes", "cheese", "cream"],
    "estimated_calories": 850}},
    {{"name": "Beef and Cheese Pasta Skillet",
    "ingredients": ["ground beef", "pasta", "tomatoes", "cheese"],
    "estimated_calories": 820
    }}
    ]
    }}

    Example Input:
    Ingredients: tomato, cucumber, red onion, black olives, feta, chicken, meat, onion, rice, bread
    Calorie Limit: 200 kcal
    Cuisines: none
    Allergies: Eggplant
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Classic Greek Salad",
    "ingredients”: ["tomato", "cucumber", "red onion", "black olives", "feta"],
    "estimated_calories": 180}},
    {{"name": "Chicken‑and‑Rice Stuffed Tomato",
    "ingredients": ["tomato", "Cooked rice", "chicken", "Onion", "black olives", "feta"],
    "estimated_calories":195}}
    ]
    }}

    Example Input:
    Ingredients: canned fish, coconut milk, rice, pasta, oats, honey, egg, butter, onion, cheese, chicken breast, lettuce, eggplant
    Calorie Limit: 400 kcal
    Cuisines: Chinese
    Allergies: non
    Dietary Preferences: Keto
    Example Output: 
    {{
    "recipes": [
    {{"name": "Eggplant and Chicken Stir-Fry",
    "ingredients": ["chicken breast", "eggplant", "onion", "butter"],
    "estimated_calories": 350}},
    {{"name": "Cheesy Egg Lettuce Wraps",
    "ingredients": "egg", "cheese", "lettuce leaves", "butter"],
    "estimated_calories":220}}
    ]
    }}

    Example Input:
    Ingredients: chicken breast, bell peppers, onions, brown rice, avocado, cheddar, salsa, tortilla, beans, cheese, sour cream, eggs, lean ground beef, lettuce, tomato
    Calorie Limit: 700 kcal
    Cuisines: Mexican
    Allergies: Egg
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Chicken Fajita Bowl",
    "ingredients": [chicken breast”, “bell peppers”, “onions”,” brown rice”, “avocado”, “cheddar”, “salsa”],
    "estimated_calories": 700}},
    {{"name": "Chicken & Bean Burrito",
    "ingredients": ["tortilla", "beans", "chicken breast", "beans", "brown rice", "cheese", "salsa",  "sour cream"],
    "estimated_calories":700}},
    {{"name": "Huevos Rancheros with Beans & Avocado",
    "ingredients": ["tortilla","avocado","salsa","cheese", "eggs", "beans"],
    "estimated_calories":650}},
    {{"name": "Taco Salad with Beef",
    "ingredients": ["lean ground beef", "lettuce", "tomato", "onions", "avocado", "cheese", "sour cream", "tortilla", "salsa"],
    "estimated_calories":700}}
    ]
    }}

    Example Input:
    Ingredients: pork belly, potatoes, butter, garlic, green beans
    Calorie Limit: 1000 kcal
    Cuisines: American
    Allergies: Dairy
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Garlic Pork Belly with Roasted Potatoes",
    "ingredients": ["pork belly", "potatoes", "garlic", "green beans"],
    "estimated_calories": 900}},
    {{"name": "Pork Belly and Potato Hash",
    "ingredients": ["pork belly", "potatoes", "garlic"],
    "estimated_calories": 870}}
    ]
    }}
        
    Example Input:
    Ingredients: chicken, rice, barberries, peanuts, saffron, onion, eggplant, kashk, garlic, mint, rice, lentils, onions, butter, eggs
    Calorie Limit: 500 kcal
    Cuisines: Iranian
    Allergies: Eggs, Peanuts
    Dietary Preferences: Gluten-free
    Example Output: 
    {{
    "recipes": [
    {{"name": "Zereshk Polo ba Morgh",
    "ingredients": ["chicken", "rice", "barberries", "saffron", "onion", "butter"],
    "estimated_calories": 500 }},
    {{"name": "Kashk-e Bademjan",
    "ingredients": ["eggplant", "onion", "kashk", "garlic", "mint"],
    "estimated_calories":480}},
    {{"name": "Adas Polo (Lentil Rice with Raisins)",
    "ingredients": ["rice", "lentils", "onions", "butter"],
    "estimated_calories":500}}
    ]
    }}


    Example Input:
    Ingredients:  chickpeas, tomato, garlic, ginger, potato, Cauliflower florets, onion, semolina, carrot, peas, beans, green chilli
    Calorie Limit: 370 kcal
    Cuisines: Indian
    Allergies: none
    Dietary Preferences: Vegan
    Example Output:
    {{
    "recipes": [
    {{
    "name": "Chana Masala with Brown Rice",
    "ingredients": ["chickpeas", garlic", "ginger", "tomato", "onion"],
    "estimated_calories": 370}},
    {{"name": "Aloo Gobi with Roti",
    "ingredients": ["potato", "Cauliflower florets", "tomato", "onion"],
    "estimated_calories": 370}},
    {{"name": "Aloo Gobi with Roti",
    "ingredients": ["semolina", "carrot", "beans","onion","green chilli"],
    "estimated_calories": 360}}
    ]
    }}

    Example Input:
    Ingredients: tofu, ginger, broccoli florets, red bell pepper, sesame seeds, cauliflower rice, bell peppers, shirataki noodles, tahini, coconut aminos, apple cider vinegar, garlic, chilli flakes, almond milk
    Calorie Limit: 700 kcal
    Cuisines: Japanese
    Allergies: Beans and Lactose
    Dietary Preferences: Keto
    Example Output:
    {{
    "recipes": [
    {{"name": "Keto Baked Tofu with Sesame-Ginger Veggies",
    "ingredients": ["tofu", "ginger", "broccoli florets", "red bell pepper", "sesame seeds", "apple cider vinegar", "olive oil"],
    "estimated_calories": 600}},
    {{"name": "Keto Sesame-Ginger Cauliflower Rice Bowl",
    "ingredients": ["cauliflower rice", "bell Peppers", "shirataki noodles", "tahini", "coconut aminos", "apple cider vinegar", "garlic", "chilli flakes", "unsweetened almond milk", "butter"],
    "estimated_calories": 650}},
    {{"name": "Spicy Shirataki Noodle Stir-Fry",
    "ingredients": ["shirataki noodles", "tofu", "red bell pepper", "broccoli florets", "garlic", "chilli flakes", "tahini", "coconut aminos", "apple cider vinegar", "olive oil", "sesame seeds"],
    "estimated_calories": 630}}
    ]
    }}


    Example Input: 
    Ingredients: zucchini, eggplant, olive oil, garlic, cherry tomatoes, basil, cauliflower, walnuts, nutritional yeast, lemon, spinach, mushrooms  
    Calorie Limit: 700 kcal  
    Cuisines: Italian 
    Allergies: Beans and Lactose 
    Dietary Preferences: Keto  
    Example Output:
    {{
    "recipes": [
    {{"name": "Keto Eggplant Parmesan (Dairy-Free)",
    "ingredients": ["eggplant", "nutritional yeast", "garlic", "basil", "olive oil", "walnuts (crushed)", "cherry tomatoes"],
    "estimated_calories": 650}},
    {{"name": "Creamy Garlic Mushroom & Zucchini Noodles",
    "ingredients": ["zucchini (spiralized)", "mushrooms", "garlic", "olive oil", "unsweetened almond milk", "nutritional yeast", "spinach", "lemon juice"],
    "estimated_calories": 600}},
   {{"name": "Walnut-Pesto Stuffed Mushrooms",
    "ingredients": ["mushrooms", "walnuts", "basil", "garlic", "olive oil", "nutritional yeast", "spinach", "lemon juice"],
    "estimated_calories": 580}}
    ]
    }}    

    Example Input:
    Ingredients: tortilla chips, beef, sausage, corn, green chilli, mushroom, cheese, milk, salami, potato, bread, chicken, butter
    Calorie Limit: 1200 kcal
    Cuisines: none
    Allergies: none
    Dietary Preferences: none
    Example Output:
    {{
    "recipes": [
    {{"name": "Loaded Nacho Mountain",
    "ingredients": ["Tortilla chips", "green chilli", "cheese", "milk", "mushroom", "corn","sausage", "beef","salami"],
    "estimated_calories": 1200}},
    {{"name": "Cheesy Meat-Stuffed Potatoes",
    "ingredients": ["Potato", "beef", "sausage", "mushroom", "milk", "cheese", "green chilli", "salami"],
    "estimated_calories": 1050}},
   {{"name": "Melty Meat Madness Sandwich",
    "ingredients": ["bread", "milk", "butter", "salami", "chicken", "sausage", "cheese", "mushroom", "green chilli"],
    "estimated_calories": 1200}}
    ]
    }}

    Ingredients: {ingredients_str}
    Calorie Limit: {calorie_limit} kcal
    Cuisines: {cuisines_str}
    Allergies: {allergies_str}
    Dietary Preferences: {diets_str}
    Example Output:
    """

    # Generate content using the client and prompt, specifying JSON response format
    response = client.generate_content(
        contents=[{"parts": [{"text": few_shot_prompt}]}],
        generation_config=genai.types.GenerationConfig(response_mime_type="application/json")
    )

    # Return the generated text (JSON)
    return response.text



## Create Interactive Food Crafter Application  
This section outlines the core user interface of the application. The layout is styled to support both light and dark modes. Based on user input, the app either processes an uploaded image to extract ingredients or allows manual entry of ingredients. Once the ingredient list is finalized, the app queries the model to generate recipe suggestions, displays recipe cards, and provides detailed breakdowns that include ingredients, instructions, nutrition information, and waste reduction tips.

In [ ]:
def create_food_crafter_app(client):
    """
    Create the interactive image upload widget with enhanced visualization
    Args:
       client 
    """
    
    # Custom CSS for styling with dark mode support
    style = widgets.HTML("""
    <style>
        /* Light mode styles */
        .custom-button { 
            background-color: #4CAF50 !important; 
            color: white !important; 
            border-radius: 8px !important; 
            padding: 15px !important; 
            font-size: 18px !important; 
            min-width: 400px !important; 
            height: 60px !important; 
            display: block !important; 
            margin: 10px auto !important; 
            transition: background-color 0.2s !important; 
        }
        .custom-button:hover { 
            background-color: #45A049 !important; 
        }
        .custom-header { 
            color: #2E7D32 !important; 
            font-family: Arial, sans-serif !important; 
            text-align: center !important; 
        }
        .section-box { 
            border: 1px solid #E0E0E0 !important; 
            padding: 10px !important; 
            border-radius: 8px !important; 
            background-color: #F9F9F9 !important; 
            margin-bottom: 10px !important; 
        }
        .recipe-card { 
            border: 1px solid #B0BEC5 !important; 
            border-radius: 8px !important; 
            padding: 15px !important; 
            margin: 10px !important; 
            background-color: #FFFFFF !important; 
            color: #000000 !important; 
            box-shadow: 2px 2px 8px rgba(0,0,0,0.1) !important; 
            transition: transform 0.2s !important; 
        }
        .recipe-card:hover { 
            transform: scale(1.02) !important; 
        }
        .loading-spinner { 
            text-align: center !important; 
            font-style: italic !important; 
            color: #616161 !important; 
        }
        .widget-label { 
            text-overflow: ellipsis !important; 
            white-space: nowrap !important; 
            overflow: visible !important; 
            min-width: 150px !important; 
            color: #000000 !important; 
        }
        .widget-checkbox .widget-label {
            color: #000000 !important;
        }
        .widget-text, .widget-dropdown {
            background-color: #FFFFFF !important;
            color: #000000 !important;
            border: 1px solid #B0BEC5 !important;
        }
        .input-message {
            margin-top: 15px !important;
            padding: 10px !important;
            background-color: #e8f5e9 !important;
            border-radius: 8px !important;
            border-left: 4px solid #4CAF50 !important;
            color: #000000 !important;
        }

        /* Dark mode styles */
        @media (prefers-color-scheme: dark) {
            .custom-button {
                background-color: #66BB6A !important;
                color: #FFFFFF !important;
            }
            .custom-button:hover {
                background-color: #81C784 !important;
            }
            .custom-header {
                color: #A5D6A7 !important;
            }
            .section-box {
                background-color: #424242 !important;
                border-color: #616161 !important;
            }
            .recipe-card {
                background-color: #616161 !important;
                color: #E0E0E0 !important;
                border-color: #757575 !important;
                box-shadow: 2px 2px 8px rgba(0,0,0,0.3) !important;
            }
            .loading-spinner {
                color: #B0BEC5 !important;
            }
            .widget-label {
                color: #E0E0E0 !important;
            }
            .widget-checkbox .widget-label {
                color: #E0E0E0 !important;
            }
            .widget-text, .widget-dropdown {
                background-color: #424242 !important;
                color: #E0E0E0 !important;
                border: 1px solid #757575 !important;
            }
            .input-message {
                background-color: #424242 !important;
                border-left: 4px solid #81C784 !important;
                color: #E0E0E0 !important;
            }
        }
    </style>
    """)

    # File upload and calorie input
    file_upload = widgets.FileUpload(
        accept='image/*',
        multiple=False,
        description='Upload Fridge Image',
        layout={'width': '400px'}
    )
    calorie_input = widgets.IntText(
        value=500,
        description='Calorie Limit (kcal):',
        layout={'width': '300px'}
    )
    upload_note = widgets.HTML(
        value='<p style="color: #2E7D32; font-style: italic;">Note: If you don’t have an image, you can manually enter your ingredients after the process.</p>'
    )
    upload_section = widgets.VBox([file_upload, calorie_input, upload_note], layout={'margin': '10px'})

    # Pantry items
    pantry_items = [
        'lentils', 'chickpeas', 'rice', 'potatoes', 'pasta', 'quinoa',
        'flour', 'sugar', 'beans', 'oats', 'cornmeal', 'butter', 'oil', 'onion', 'garlic'
    ]
    # Create checkboxes for each pantry item
    pantry_checkboxes = [
        widgets.Checkbox(value=False, description=item, indent=False, layout={'width': '200px'})
        for item in pantry_items
    ]
    # Arrange checkboxes in a grid layout
    pantry_grid = widgets.GridBox(
        pantry_checkboxes,
        layout={'grid_template_columns': 'repeat(3, 200px)'}
    )
    pantry_label = widgets.HTML(value="<b>Select Pantry Items</b>")

    # Manually adding custom pantry items
    custom_pantry_input = widgets.Text(
        value='',
        placeholder='e.g., canned tomatoes, soy sauce',
        description='Custom Pantry Items:',
        layout={'width': '600px'}
    )
    # Combine pantry widgets
    pantry_section = widgets.VBox([pantry_label, pantry_grid, custom_pantry_input], layout={'margin': '10px'})

    
    # Cuisine preferences
    cuisines = ['Italian', 'Chinese', 'Mexican', 'Indian', 'Japanese', 'American', 'Mediterranean']
    # Create checkboxes for each cuisine item
    cuisine_checkboxes = [
        widgets.Checkbox(value=False, description=cuisine, indent=False, layout={'width': '200px'})
        for cuisine in cuisines
    ]
    # Arrange checkboxes in a grid layout
    cuisine_grid = widgets.GridBox(
        cuisine_checkboxes,
        layout={'grid_template_columns': 'repeat(3, 200px)'}
    )
    cuisine_label = widgets.HTML(value="<b>Select Preferred Cuisines</b>")
    cuisine_section = widgets.VBox([cuisine_label, cuisine_grid], layout={'margin': '10px'})

    
    # Allergies list
    allergies = ['Lactose', 'Nuts', 'Shellfish', 'Eggs', 'Soy', 'Fish']
    # Create checkboxes for each allergy
    allergy_checkboxes = [
        widgets.Checkbox(value=False, description=allergy, indent=False, layout={'width': '200px'})
        for allergy in allergies
    ]
    allergy_grid = widgets.GridBox(
        allergy_checkboxes,
        layout={'grid_template_columns': 'repeat(3, 200px)'}
    )
    allergy_label = widgets.HTML(value="<b>Select Allergies</b>")
    # Manually adding custom allergies
    custom_allergy_input = widgets.Text(
        value='',
        placeholder='e.g., sesame, celery',
        description='Other Allergies:',
        layout={'width': '600px'}
    )
    # Combine allergy widgets
    allergy_section = widgets.VBox([allergy_label, allergy_grid, custom_allergy_input], layout={'margin': '10px'})

    
    # Dietary preferences
    diets = ['Vegetarian', 'Vegan', 'Pescatarian', 'Keto', 'Gluten-Free']
    # Create checkboxes for each allergy
    diet_checkboxes = [
        widgets.Checkbox(value=False, description=diet, indent=False, layout={'width': '200px'})
        for diet in diets
    ]
    diet_grid = widgets.GridBox(
        diet_checkboxes,
        layout={'grid_template_columns': 'repeat(3, 200px)'}
    )
    diet_label = widgets.HTML(value="<b>Select Dietary Preferences</b>")
    # Manually adding custom diet
    custom_diets_input = widgets.Text(
        value='',
        placeholder='e.g., Low-Carb, Paleo',
        description='Other Dietary :',
        layout={'width': '600px'}
    )
    # Combine diet widgets
    diet_section = widgets.VBox([diet_label, diet_grid, custom_diets_input], layout={'margin': '10px'})

    # Accordion for input sections
    accordion = widgets.Accordion(children=[
        upload_section,
        pantry_section,
        cuisine_section,
        allergy_section,
        diet_section
    ])
    accordion.set_title(0, 'Image Upload & Calorie Limit')
    accordion.set_title(1, 'Pantry Items')
    accordion.set_title(2, 'Cuisine Preferences')
    accordion.set_title(3, 'Allergies')
    accordion.set_title(4, 'Dietary Preferences')

    # Process button
    process_button = widgets.Button(
        description='Process',
        button_style='',
        tooltip='Click to identify ingredients and suggest recipes',
        layout={'width': '100%', 'margin': '10px auto', 'display': 'block'}
    )
    process_button.add_class('custom-button')

    # Create separate output areas
    image_output = widgets.Output()
    main_output = widgets.Output()

    # Global variable for storing generated recipes
    recipes_json_global = [None]

    def on_button_click(b):
        # Clear only the main output but keep the image output intact
        main_output.clear_output()
        image_output.clear_output()
        
        all_ingredients = []

        # Check if an image is uploaded
        if file_upload.value:
            with image_output:
                uploaded_file_info = file_upload.value[0]
                image_bytes = uploaded_file_info['content']
                display(Markdown("### Uploaded Fridge Image"))
                display(display_image(image_bytes))
            
            # Process the image and extract ingredients
            with main_output:
                display(widgets.HTML('<div class="loading-spinner">Processing image ingredients...</div>'))
                try:
                    image_ingredients = identify_ingredients_from_image(client, image_bytes)
                    all_ingredients.extend(image_ingredients)
                    main_output.clear_output()
                    display(Markdown("### Ingredients from Image"))
                    for i, ingredient in enumerate(image_ingredients, 1):
                        print(f"{i}. {ingredient}")
                    
                    # Create a button to allow adding manual ingredients
                    add_ingredients_button = widgets.Button(
                        description='Add Manual Ingredients',
                        button_style='',
                        tooltip='Click to add more ingredients manually',
                        layout={'width': 'auto', 'margin': '10px'}
                    )
                    add_ingredients_button.add_class('custom-button')
                    
                    # Output area for additional manual input
                    additional_input_output = widgets.Output()
                    
                    def on_add_ingredients_clicked(b):
                        """
                        Handle click on "Add Manual Ingredients" button:
                        - Display a text field for the user to enter missing ingredients.
                        - Show a button to submit and add those ingredients.
                        """
                        with additional_input_output:
                            additional_input_output.clear_output()
                            display(widgets.HTML(
                                value='<div class="input-message"><b>If anything’s missing or not recognized, feel free to add it yourself!</b><br>Just enter your ingredients below, separated by commas (for example: tomatoes, spinach, chicken):</div>'
                            ))
                            additional_input = widgets.Text(
                                value='',
                                placeholder='e.g., mozzarella balls, orange bell pepper, spinach',
                                description='',
                                layout={'width': '600px'}
                            )
                            submit_additional_button = widgets.Button(
                                description='Submit Additional Ingredients',
                                button_style='',
                                layout={'width': 'auto', 'margin': '10px'}
                            )
                            submit_additional_button.add_class('custom-button')
                            
                            def on_submit_additional_clicked(b):
                                """Handle adding manual ingredients"""
                                with additional_input_output:
                                    additional_input_output.clear_output()
                                    if additional_input.value.strip():
                                        new_ingredients = [item.strip() for item in additional_input.value.split(',') if item.strip()]
                                        all_ingredients.extend(new_ingredients)
                                        display(Markdown("### Additional Ingredients Added"))
                                        for i, item in enumerate(new_ingredients, 1):
                                            print(f"{i}. {item}")
                                    # Proceed to process all ingredients
                                    process_all_ingredients(all_ingredients)
                            
                            submit_additional_button.on_click(on_submit_additional_clicked)
                            display(additional_input, submit_additional_button)
                    
                    add_ingredients_button.on_click(on_add_ingredients_clicked)
                    display(add_ingredients_button, additional_input_output)
                
                except Exception as e:
                    main_output.clear_output()
                    print(f"Error identifying image ingredients: {str(e)}")
                    return
        
        else:
            # No image uploaded, prompt for manual ingredient input
            with main_output:
                display(widgets.HTML(
                    value='<div class="input-message"><b>No image? No problem!</b><br>Just enter your ingredients below, separated by commas (for example: tomatoes, spinach, chicken):</div>'
                ))
                manual_input = widgets.Text(
                    value='',
                    placeholder='e.g., tomatoes, spinach, chicken',
                    description='Ingredients:',
                    layout={'width': '600px'}
                )
                submit_manual_button = widgets.Button(
                    description='Submit Manual Ingredients',
                    button_style='',
                    layout={'width': 'auto', 'margin': '10px'}
                )
                submit_manual_button.add_class('custom-button')
                manual_input_output = widgets.Output()
                
                def on_submit_manual_clicked(b):
                    """Handle adding manual ingredients: update list, display entries, and proceed."""
                    with manual_input_output:
                        manual_input_output.clear_output()
                        if manual_input.value.strip():
                            manual_ingredients = [item.strip() for item in manual_input.value.split(',') if item.strip()]
                            all_ingredients.extend(manual_ingredients)
                            display(Markdown("### Manually Entered Ingredients"))
                            for i, item in enumerate(manual_ingredients, 1):
                                print(f"{i}. {item}")
                            # Proceed to process all ingredients
                            process_all_ingredients(all_ingredients)
                        else:
                            # Allow proceeding with pantry items even if no manual ingredients are entered
                            process_all_ingredients(all_ingredients)
                
                submit_manual_button.on_click(on_submit_manual_clicked)
                display(manual_input, submit_manual_button, manual_input_output)

    def process_all_ingredients(all_ingredients):
        """
        Gather and display all user-provided ingredients and preferences, then generate and show recipe suggestions.
        - Add selected pantry and any custom pantry items to `all_ingredients`, displaying each.
        - Display chosen cuisines, allergies/intolerances, and dietary preferences.
        - If no ingredients are available, prompt the user and exit.
        - Fetch recipe suggestions under the calorie limit, show structured JSON and render recipe cards.
        - Provide controls to select a recipe and fetch detailed preparation steps.
        """
        with main_output:
            # Collect pantry items
            pantry_selected = [cb.description for cb in pantry_checkboxes if cb.value]
            if pantry_selected:
                all_ingredients.extend(pantry_selected)
                display(Markdown("### Pantry Items Selected"))
                for i, item in enumerate(pantry_selected, 1):
                    print(f"{i}. {item}")

            custom_pantry_items = []
            if custom_pantry_input.value.strip():
                custom_pantry_items = [item.strip() for item in custom_pantry_input.value.split(',') if item.strip()]
                all_ingredients.extend(custom_pantry_items)
                if custom_pantry_items:
                    display(Markdown("### Custom Pantry Items Added"))
                    for i, item in enumerate(custom_pantry_items, 1):
                        print(f"{i}. {item}")

            # Collect cuisines, allergies, and diets
            selected_cuisines = [cb.description for cb in cuisine_checkboxes if cb.value]
            if selected_cuisines:
                display(Markdown("### Selected Cuisines"))
                for i, cuisine in enumerate(selected_cuisines, 1):
                    print(f"{i}. {cuisine}")
            else:
                selected_cuisines = None

            selected_allergies = [cb.description for cb in allergy_checkboxes if cb.value]
            if custom_allergy_input.value.strip():
                custom_allergies = [allergy.strip() for allergy in custom_allergy_input.value.split(',') if allergy.strip()]
                selected_allergies.extend(custom_allergies)
            if selected_allergies:
                display(Markdown("### Allergies/Intolerances"))
                for i, allergy in enumerate(selected_allergies, 1):
                    print(f"{i}. {allergy}")

            selected_diets = [cb.description for cb in diet_checkboxes if cb.value]
            if custom_diets_input.value.strip():
                custom_diets = [diet.strip() for diet in custom_diets_input.value.split(',') if diet.strip()]
                selected_diets.extend(custom_diets)
            if selected_diets:
                display(Markdown("### Dietary Preferences"))
                for i, diet in enumerate(selected_diets, 1):
                    print(f"{i}. {diet}")

            # Check if all_ingredients is empty after adding pantry items
            if not all_ingredients:
                print("No ingredients provided. Please upload an image, select pantry items, or enter custom ingredients.")
                return

            calorie_limit = calorie_input.value
            display(Markdown(f"### Recipe Suggestions (max {calorie_limit} calories)"))
            display(widgets.HTML('<div class="loading-spinner">Generating recipe suggestions...</div>'))
            try:
                recipes_json_global[0] = suggest_recipes_from_ingredients(
                    client, all_ingredients, calorie_limit, selected_cuisines, selected_allergies, selected_diets
                )
                display(Markdown("### Structured Recipe Output"))
                display(Markdown(f"```json\n{recipes_json_global[0]}\n```"))

                try:
                    suggested_recipes = json.loads(recipes_json_global[0])['recipes']
                    #print(f"show **{recipe}")#delete
                    if suggested_recipes:
                        # Display recipes as cards
                        recipe_cards = []
                        for recipe in suggested_recipes:
                            card = widgets.HTML(f"""
                            <div class="recipe-card">
                                <h3>{recipe['name']}</h3>
                                <p><b>Ingredients:</b> {', '.join(recipe['ingredients'])}</p>
                                <p><b>Calories:</b> {recipe['estimated_calories']} kcal</p>
                            </div>
                            """)
                            recipe_cards.append(card)
                        display(widgets.VBox(recipe_cards))

                        # Recipe selection for details
                        recipe_options = [recipe['name'] for recipe in suggested_recipes]
                        select_recipe_label = widgets.Label("Select a recipe to see details:")
                        recipe_selection = widgets.Dropdown(
                            options=recipe_options,
                            description='Choose Recipe:',
                            disabled=False
                        )
                        show_details_button = widgets.Button(
                            description="Show Recipe Details",
                            button_style='',
                            layout={'width': 'auto'}
                        )
                        show_details_button.add_class('custom-button')
                        details_output = widgets.Output()

                        def on_show_details_clicked(b):
                            """Show detailed information for the selected recipe"""
                            with details_output:
                                details_output.clear_output()
                                selected_name = recipe_selection.value
                                for recipe in suggested_recipes:
                                    if recipe['name'] == selected_name:
                                        display(Markdown(f"### Detailed Information for: **{recipe['name']}**"))
                                        display(Markdown("**Ingredients:**"))
                                        print("- " + "\n- ".join(recipe['ingredients']))
                                        display(Markdown(f"**Estimated Calories:** {recipe['estimated_calories']}"))

                                        
                                        # Prepare a prompt for more details
                                        cuisines_text = ', '.join(selected_cuisines) if selected_cuisines else 'any'
                                        allergies_text = ', '.join(selected_allergies) if selected_allergies else 'none'
                                        diets_text = ', '.join(selected_diets) if selected_diets else 'none'

                                        
                                        prompt = f"""Provide a more detailed description and potential preparation steps for the recipe: '{selected_name}'.
                                        Consider the following constraints:
                                        - Calorie Limit: {recipe['estimated_calories']} kcal
                                        - Preferred cuisines: {cuisines_text}
                                        - Allergies to avoid: {allergies_text}
                                        - Dietary preferences: {diets_text}
                                        - Only consider these Ingredients: {recipe['ingredients']}
                                        Assume the user has common spices available (e.g., salt, black pepper, red pepper, garlic powder, onion powder,paprika,basic herbs).

                                        Include these EXACT sections:

                                        **INGREDIENTS:**
                                        - Only list ingredients that were provided by the user.
                                        - Standard measurements
                                        - Include precise quantities (e.g., 1/2 cup, 2 tablespoons, 1 tsp).

                                        **INSTRUCTIONS:**
                                        - Numbered steps
                                        - Cooking times

                                        **WASTE REDUCTION:**
                                        - Specific actionable tips

                                        **NUTRITION:**
                                        - Per serving
                                        - Key nutrients
                                        
                                        **PERSONALIZATION NOTES:**
                                        - Briefly explain how this recipe aligns with the user's selected cuisines, allergies, and dietary preferences, based ONLY on the information provided.

                                        """    
                                        
                                        display(widgets.HTML('<div class="loading-spinner">Fetching details...</div>'))
                                        try:
                                            response = client.generate_content(
                                                contents=[{"parts": [{"text": prompt}]}]
                                            )
                                            details_output.clear_output()
                                            display(Markdown("**More Details:**"))
                                            print(response.text.strip())
                                        except Exception as e:
                                            details_output.clear_output()
                                            print(f"Error fetching detailed recipe information: {e}")
                                        return
                                display(Markdown("Recipe details not found."))

                        show_details_button.on_click(on_show_details_clicked)

                        display(select_recipe_label, recipe_selection, show_details_button, details_output)

                    else:
                        print("No recipes found in the suggestions to select from.")

                except (json.JSONDecodeError, KeyError) as e:
                    print(f"Error processing recipe suggestions for selection: {e}")

            except Exception as e:
                print(f"Error generating recipes: {str(e)}")

    process_button.on_click(on_button_click)

    # App layout with separate image and main output areas
    header = widgets.HTML(value='<h1 class="custom-header">Waste and Calorie Wise Food Crafter</h1>')
    description = widgets.HTML(value='<p style="color: #008000; font-weight: bold;">Upload an image of your fridge (or manually enter everything), select pantry items, choose cuisines, specify allergies and diets, and get tailored recipe suggestions.</p>')

    app = widgets.VBox([
        style,
        header,
        description,
        widgets.HTML('<div class="section-box">'),
        accordion,
        process_button,
        widgets.HTML('</div>'),
        image_output,
        main_output
    ])

    return app

##  Run and Configuration
Configure the Google Generative AI client using securely stored API keys in Kaggle Secrets. The app is launched with all interactive components ready for user input.



In [ ]:
def run_food_crafter_app():
    """Main execution code with improved error handling"""
    # Check API key
    try:
        from kaggle_secrets import UserSecretsClient
        GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    except ImportError:
        print("Warning: kaggle_secrets not found. Using manual API key.")
        GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY"  # Replace with your actual API key
        if GOOGLE_API_KEY == "YOUR_GOOGLE_API_KEY":
            print("ERROR: Please replace 'YOUR_GOOGLE_API_KEY' with your actual Google API Key.")
            return
    except Exception as e:
        print(f"Error retrieving API key: {e}")
        return
    
    # Initialize Google Generative AI
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
    except Exception as e:
        print(f"Error configuring Google Generative AI: {e}")
        return
    
    # Create client
    try:
        client = genai.GenerativeModel('gemini-2.0-flash')
    except Exception as e:
        print(f"Error creating Gemini model client: {e}")
        print("This could be due to an invalid API key or the Gemini-2.0-flash model not being available.")
        return
    
    # Create and display app
    try:
        app = create_food_crafter_app(client)
        display(app)
    except Exception as e:
        print(f"Error creating or displaying app: {e}")
        print(f"Error details: {type(e).__name__}: {str(e)}")
        
        # Additional debugging
        import traceback
        traceback.print_exc()

# Run the app
run_food_crafter_app()

# Conclusion
BiteWise, an AI-powered personalized meal planning tool, effectively demonstrates its ability to assist users make informed, healthy, and low-waste meal choices based on their preferences and the ingredients they have on hand. By combining calorie awareness, dietary customization, cuisine preferences, pantry selection, image-based or manual ingredient input, and structured recipe generation, the system provides a practical solution for everyday meal planning.

# Future work
* **Grocery receipt use:** Scan shopping receipts to auto-fill pantry items.
* **Real-Time Grounded Model Calls:** Link to live web data to recognize new foods, and updated nutrition info without retraining.
* **Improved Image Recognition:** Fine‑tune the vision model to handle messy shelves, mixed lighting, and smaller items.
* **AI chat interface:** Let users adjust recipes, ask for alternatives, or get cooking tips through conversation.
* **Voice commands:** Enable hands-free interaction (e.g., “Hey AI, what’s for dinner?”).
* **Smart grocery list integration:** Automatically add missing ingredients to a shopping cart or list.

# Team Credits
BiteWise was created as part of the 2025Q1 Capstone Project for Google’s Generative AI Intensive Course.

**Our team**: Mohammad Mahdi Heydari asl, Paniz Oghabi, Shima Rastegarnia.